In [74]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [75]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [76]:
#viewing test and train datasets from the csv files
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [77]:
test_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,67991,67991,40000.0,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,...,97.7,0.0,0.0,0.0,527975.0,70914.0,74600.0,99475.0,N,N
1,25429,25429,6000.0,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,...,66.7,0.0,0.0,0.0,34628.0,23460.0,5900.0,23628.0,N,N
2,38496,38496,3600.0,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,...,100.0,0.0,0.0,0.0,23100.0,19183.0,7300.0,15000.0,N,N
3,19667,19667,20000.0,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,56481.0,43817.0,13800.0,35981.0,N,N
4,37505,37505,3600.0,0.1240,120.27,RENT,50000.0,Not Verified,low_risk,n,...,100.0,25.0,0.0,0.0,45977.0,32448.0,21000.0,24977.0,N,N


In [78]:
# look for the features
test_df.columns

Index(['Unnamed: 0', 'index', 'loan_amnt', 'int_rate', 'installment',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_ol

In [79]:
# solve issue of mismatch in train and test set after categorical encoding
# function to check unmatching columns

def check_missing_columns(train_df,test_df):
    # Display warning if columns do not match
    inner = set(train_df.columns) & set(test_df.columns)
    full = set(train_df.columns) | set(test_df.columns)
    unmatch = list(full - inner)

    if (len(unmatch) != 0):
        print("columns count does not match")
        return unmatch
    else:
        print("columns count matched")

In [80]:
check_matching_columns(train_df,test_df)

columns count matched


In [81]:

# Dummy Encoding --Convert categorical data to numeric and separate target feature for training data
# Convert categorical data to numeric and separate target feature for testing data
# add missing dummy variables to testing set-- 
# separating features

train_df = pd.get_dummies(train_df, drop_first = True)
X_train = train_df.drop(columns=['loan_status_low_risk'])
y_train = train_df['loan_status_low_risk'].values


In [82]:
X_train.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App,hardship_flag_Y,debt_settlement_flag_Y
0,57107,57107,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,...,170200.0,1,0,0,0,0,1,0,0,0
1,141451,141451,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,...,35398.0,1,0,0,1,0,1,0,0,0
2,321143,321143,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,...,90340.0,1,0,0,1,0,1,0,0,0
3,11778,11778,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,...,15406.0,0,0,1,0,0,1,0,0,0
4,169382,169382,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,...,58778.0,1,0,0,1,0,1,0,0,0


In [83]:
#Testing Data
test_df = pd.get_dummies(test_df, drop_first = True)
X_test = test_df.drop(columns=['loan_status_low_risk'])
y_test = test_df['loan_status_low_risk'].values


In [84]:
# after conversion 
X_test.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App,hardship_flag_Y
0,67991,67991,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,...,74600.0,99475.0,1,0,0,0,0,1,0,0
1,25429,25429,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,...,5900.0,23628.0,0,0,1,0,0,1,0,0
2,38496,38496,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,...,7300.0,15000.0,0,0,1,0,0,1,0,0
3,19667,19667,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,...,13800.0,35981.0,0,0,1,0,0,1,0,0
4,37505,37505,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,...,21000.0,24977.0,0,0,1,0,0,1,0,0


In [85]:
#checking the missing column

check_missing_columns(X_train,X_test)


columns count does not match


['debt_settlement_flag_Y']

In [88]:
# Fill in the blank with dummy variables
# add missing dummy variables to testing set
X_test['debt_settlement_flag_Y'] =0

check_matching_columns(X_train,X_test)

columns count matched


In [89]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(12180, 87) (12180,)
(4702, 87) (4702,)


In [98]:
# Train the Logistic Regression model on the unscaled data and print the model score

logistic_regression_model = LogisticRegression(max_iter = 50000, random_state=5).fit(X_train,y_train)
print("Logistic Regression score: ", logistic_regression_model.score(X_test, y_test))

Logistic Regression score:  0.5682688217779668


/Users/jennybui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [92]:
# Train a Random Forest Classifier model and print the model score
random_model = RandomForestClassifier(n_estimators=50, random_state = 5).fit(X_train,y_train)
print("Random Forest Classifier score: ", random_model.score(X_test, y_test))

Random Forest Classifier score:  0.6210123351765207


In [94]:
# Scale the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
Make a prediction as to which model you think will perform better for scaled data:
I still think Random forest classifier will do better based on the decision tree theory. It will do a better at overfitting our scaled data.

In [95]:
# Train the Logistic Regression model on the scaled data and print the model score
logistic_scaled_model = LogisticRegression(max_iter = 35000, random_state=5).fit(X_train_scaled,y_train)
print("Logistic Regression Scaled score: ", logistic_scaled_model.score(X_test_scaled, y_test))

Logistic Regression Scaled score:  0.7201190982560612


In [96]:
# Train a Random Forest Classifier model on the scaled data and print the model score

random_scaled_model = RandomForestClassifier(random_state = 5, n_estimators=50).fit(X_train_scaled,y_train)
print("Random Forest Classifier Scaled score: ", random_scaled_model.score(X_test_scaled, y_test))

Random Forest Classifier Scaled score:  0.6205869842620162


In [ ]:
In Conclusion:
Both models showcased similar score results. There was an issue of iteration score for unscaled data that used linear regression model. This produced a score of 0.56 which is relatively low. Next time, I would either increase the max iteration or scale the data to fit.
For the scaled data logistic regression works better by a difference of 0.10 than random forest classifier. As for unscaled data random forest works better.
For Random forest, scaled data was expected to have a higher score but it didn't perform well as expected. 